# Test Environment

In [5]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
#import Models.models as models
#import Models.LSTM.models_LSTM as models
import myLibrary as mL
#from Experiment_Class import Experiment
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import os

NDBC = mL.NDBC_lib
ERA5 = mL.ERA5_lib
Models = mL.Models
DP = mL.DataProcessor
Experiment = mL.Experiment

# Get Data

In [6]:
data_directory = os.path.join(os.getcwd(), f'data/datasets/type_B')

def build_UI():

    # Select Model-------------------------------------------------------------------------------
    global datafile_widget
    datafile_list = os.listdir(data_directory)
    datafile_widget = widgets.Select(
        options=datafile_list,
        value=datafile_list[0],
        # rows=10,
        description='Datafile:',
        disabled=False
    )
    display(datafile_widget)

    #STATIONARY_SHIFT -----------------------------------------------------------------------
    global stationaryShift_widget
    stationaryShift_widget = widgets.BoundedIntText(
        value=1,
        min=0,
        max=10,
        step=1,
        description='',
        disabled=False,
    )
    print("Stationary Shilft: ")
    display(stationaryShift_widget)


    # Select Model-------------------------------------------------------------------------------
    global models_widget
    models_list = list(Models.model_dictionary.keys())
    models_widget = widgets.Select(
        options=models_list,
        value=models_list[0],
        # rows=10,
        description='Model:',
        disabled=False
    )
    display(models_widget)

    #ALPHA:-------------------------------------------------------------------------------
    # create a FloatSlider widget for a value between 0 and 1
    print("Alpha (only for PINN):")
    global alpha_slider
    alpha_slider = widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.01,
        description='',
        readout_format='.2f',
        orientation='horizontal',
        layout={'width': '500px'}
    )

    # display the FloatSlider widget
    display(alpha_slider)

#build_UI()

In [7]:
#DATAFILE = datafile_widget.value
#print(DATAFILE)
# STATIONARY_SHIFT = stationaryShift_widget.value
# MODEL_NAME = models_widget.value
# ALPHA = alpha_slider.value

## Optional: use hardcoded variables instead

In [8]:
DATAFILE = "dataset_GOM_1_B_B.pickle"
STATIONARY_SHIFT = 1
MODEL_NAME = "LSTM"
ALPHA = 0

In [9]:
print(f"Datafile: {DATAFILE}")
print(f"Stationary Shift: {STATIONARY_SHIFT}")
print(f"Model: {MODEL_NAME}")
print(f"Alpha: {ALPHA}")

Datafile: dataset_GOM_1_B_B.pickle
Stationary Shift: 1
Model: LSTM
Alpha: 0


In [10]:
with open(f'data/datasets/type_B/{DATAFILE}', 'rb') as f:
    # load the object from the file using pickle.load()
    dataset = pickle.load(f)

print("files:", dataset["files"])
print("nan_threshold", dataset["nan_threshold"])
print("features", dataset["features"])
print("add_era5", dataset["add_era5"])
print("num_test_hours", dataset["num_test_hours"])

train = dataset["data_train"]
test = dataset["data_test"]

display(train)
display(test)

files: [['42001', '2002'], ['42002', '2002'], ['42003', '2002'], ['42007', '2002'], ['42012', '2002'], ['42019', '2002'], ['42020', '2002'], ['42035', '2002'], ['42036', '2002'], ['42038', '2002'], ['42039', '2002'], ['42040', '2002'], ['42041', '2002'], ['42055', '2002'], ['42001', '2003'], ['42002', '2003'], ['42003', '2003'], ['42007', '2003'], ['42012', '2003'], ['42019', '2003'], ['42020', '2003'], ['42035', '2003'], ['42036', '2003'], ['42038', '2003'], ['42039', '2003'], ['42040', '2003'], ['42041', '2003'], ['42055', '2003'], ['42001', '2004'], ['42002', '2004'], ['42003', '2004'], ['42007', '2004'], ['42012', '2004'], ['42019', '2004'], ['42020', '2004'], ['42035', '2004'], ['42036', '2004'], ['42038', '2004'], ['42039', '2004'], ['42040', '2004'], ['42041', '2004'], ['42055', '2004'], ['42001', '2005'], ['42002', '2005'], ['42003', '2005'], ['42007', '2005'], ['42012', '2005'], ['42019', '2005'], ['42020', '2005'], ['42035', '2005'], ['42036', '2005'], ['42038', '2005'], ['42

,WDIR(t-3),WSPD(t-3),PRES(t-3),WDIR_ERA5(t-3),WSPD_ERA5(t-3),PRES_ERA5(t-3),WDIR(t-2),WSPD(t-2),PRES(t-2),WDIR_ERA5(t-2),...,PRES(t-1),WDIR_ERA5(t-1),WSPD_ERA5(t-1),PRES_ERA5(t-1),WDIR(t),WSPD(t),PRES(t),WDIR_ERA5(t),WSPD_ERA5(t),PRES_ERA5(t)
2002-01-01 04:00:00,66.0,9.3,1017.1,198.460880,9.782997,1016.800242,67.0,9.4,1017.2,195.622229,...,1017.2,192.752865,8.911373,1016.562914,70.0,9.0,1017.1,192.963684,8.481407,1017.104074
2002-01-01 05:00:00,67.0,9.4,1017.2,195.622229,9.517146,1016.907822,69.0,9.1,1017.2,192.752865,...,1017.1,192.963684,8.481407,1017.104074,69.0,9.6,1016.5,194.397209,8.035067,1016.824366
2002-01-01 06:00:00,69.0,9.1,1017.2,192.752865,8.911373,1016.562914,70.0,9.0,1017.1,192.963684,...,1016.5,194.397209,8.035067,1016.824366,72.0,9.5,1016.2,193.785241,8.451408,1016.893478
2002-01-01 07:00:00,70.0,9.0,1017.1,192.963684,8.481407,1017.104074,69.0,9.6,1016.5,194.397209,...,1016.2,193.785241,8.451408,1016.893478,70.0,9.6,1016.3,194.463238,8.955747,1016.710266
2002-01-01 13:00:00,78.0,10.2,1015.7,191.828712,9.656415,1015.942210,80.0,10.0,1015.7,184.076283,...,1014.9,175.759396,10.050993,1015.906350,90.0,10.2,1015.2,166.023397,9.709110,1016.105210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 20:00:00,170.0,7.6,1016.1,96.994334,7.125562,1014.631331,176.0,6.5,1015.5,92.810490,...,1014.1,88.714044,6.862280,1012.748607,187.0,5.7,1013.8,84.951047,6.433312,1012.007058
2022-12-30 21:00:00,176.0,6.5,1015.5,92.810490,6.996244,1013.912606,177.0,7.0,1014.1,88.714044,...,1013.8,84.951047,6.433312,1012.007058,180.0,5.8,1013.0,82.582222,5.941144,1011.541975
2022-12-30 22:00:00,177.0,7.0,1014.1,88.714044,6.862280,1012.748607,187.0,5.7,1013.8,84.951047,...,1013.0,82.582222,5.941144,1011.541975,178.0,5.0,1012.9,79.159604,4.810049,1011.839973
2022-12-30 23:00:00,187.0,5.7,1013.8,84.951047,6.433312,1012.007058,180.0,5.8,1013.0,82.582222,...,1012.9,79.159604,4.810049,1011.839973,174.0,4.2,1013.3,78.113716,4.049553,1011.913611


,WDIR(t-3),WSPD(t-3),PRES(t-3),WDIR_ERA5(t-3),WSPD_ERA5(t-3),PRES_ERA5(t-3),WDIR(t-2),WSPD(t-2),PRES(t-2),WDIR_ERA5(t-2),...,PRES(t-1),WDIR_ERA5(t-1),WSPD_ERA5(t-1),PRES_ERA5(t-1),WDIR(t),WSPD(t),PRES(t),WDIR_ERA5(t),WSPD_ERA5(t),PRES_ERA5(t)
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-12-31 02:00:00,185.0,5.9,1012.9,75.851396,5.301299,1013.856175,189.0,5.0,1013.5,66.884523,...,1013.8,67.669663,5.587926,1015.191389,169.0,2.3,1014.5,72.227735,5.840792,1015.804887
2022-12-31 03:00:00,189.0,5.0,1013.5,66.884523,5.057694,1014.403047,203.0,5.6,1013.8,67.669663,...,1014.5,72.227735,5.840792,1015.804887,53.0,6.0,1014.8,69.389256,6.463279,1016.208879
2022-12-31 04:00:00,203.0,5.6,1013.8,67.669663,5.587926,1015.191389,169.0,2.3,1014.5,72.227735,...,1014.8,69.389256,6.463279,1016.208879,55.0,8.5,1015.2,61.461657,6.500897,1016.314402
2022-12-31 05:00:00,169.0,2.3,1014.5,72.227735,5.840792,1015.804887,53.0,6.0,1014.8,69.389256,...,1015.2,61.461657,6.500897,1016.314402,72.0,7.6,1014.9,53.576686,6.541557,1016.235452
2022-12-31 06:00:00,53.0,6.0,1014.8,69.389256,6.463279,1016.208879,55.0,8.5,1015.2,61.461657,...,1014.9,53.576686,6.541557,1016.235452,77.0,7.2,1014.4,56.308269,6.465415,1015.708607
2022-12-31 07:00:00,55.0,8.5,1015.2,61.461657,6.500897,1016.314402,72.0,7.6,1014.9,53.576686,...,1014.4,56.308269,6.465415,1015.708607,97.0,7.8,1014.1,61.932861,6.487022,1015.506803
2022-12-31 08:00:00,72.0,7.6,1014.9,53.576686,6.541557,1016.235452,77.0,7.2,1014.4,56.308269,...,1014.1,61.932861,6.487022,1015.506803,148.0,3.3,1013.9,65.444207,6.690521,1015.204484
2022-12-31 09:00:00,77.0,7.2,1014.4,56.308269,6.465415,1015.708607,97.0,7.8,1014.1,61.932861,...,1013.9,65.444207,6.690521,1015.204484,195.0,5.4,1014.0,60.655439,6.661433,1015.499486
2022-12-31 10:00:00,97.0,7.8,1014.1,61.932861,6.487022,1015.506803,148.0,3.3,1013.9,65.444207,...,1014.0,60.655439,6.661433,1015.499486,223.0,5.1,1014.4,44.785787,4.087878,1015.262637


# Data Processing

## 1. Make data stationary

In [11]:
STATIONARY = True #Set Flag for report
train_stationary = DP.data_to_stationary(train, n = STATIONARY_SHIFT)
test_stationary = DP.data_to_stationary(test, n = STATIONARY_SHIFT)

print(f"TRAIN: shape: {train_stationary.shape}")
display(train_stationary.head())

print(f"TEST: shape: {test_stationary.shape}")
display(test_stationary.head())

TRAIN: shape: (832867, 24)


,WDIR(t-3),WSPD(t-3),PRES(t-3),WDIR_ERA5(t-3),WSPD_ERA5(t-3),PRES_ERA5(t-3),WDIR(t-2),WSPD(t-2),PRES(t-2),WDIR_ERA5(t-2),...,PRES(t-1),WDIR_ERA5(t-1),WSPD_ERA5(t-1),PRES_ERA5(t-1),WDIR(t),WSPD(t),PRES(t),WDIR_ERA5(t),WSPD_ERA5(t),PRES_ERA5(t)
2002-01-01 05:00:00,1.0,0.1,0.1,-2.838651,-0.265851,0.107580,2.0,-0.3,0.0,-2.869363,...,-0.1,0.210819,-0.429966,0.541160,-1.0,0.6,-0.6,1.433525,-0.446340,-0.279708
2002-01-01 06:00:00,2.0,-0.3,0.0,-2.869363,-0.605774,-0.344908,1.0,-0.1,-0.1,0.210819,...,-0.6,1.433525,-0.446340,-0.279708,3.0,-0.1,-0.3,-0.611968,0.416342,0.069112
2002-01-01 07:00:00,1.0,-0.1,-0.1,0.210819,-0.429966,0.541160,-1.0,0.6,-0.6,1.433525,...,-0.3,-0.611968,0.416342,0.069112,-2.0,0.1,0.1,0.677998,0.504338,-0.183212
2002-01-01 13:00:00,8.0,1.2,-1.4,-1.134972,1.175009,-1.161864,11.0,0.4,-0.8,-10.320926,...,-1.3,-18.025845,1.599584,-0.987128,20.0,0.6,-1.1,-28.439841,0.753363,-0.605056
2002-01-01 14:00:00,2.0,-0.2,0.0,-7.752430,0.408429,0.158436,7.0,0.7,-0.8,-8.316887,...,0.3,-9.735999,-0.341883,0.198860,-1.0,0.6,0.4,-5.044216,-0.446621,0.691772


TEST: shape: (21, 24)


,WDIR(t-3),WSPD(t-3),PRES(t-3),WDIR_ERA5(t-3),WSPD_ERA5(t-3),PRES_ERA5(t-3),WDIR(t-2),WSPD(t-2),PRES(t-2),WDIR_ERA5(t-2),...,PRES(t-1),WDIR_ERA5(t-1),WSPD_ERA5(t-1),PRES_ERA5(t-1),WDIR(t),WSPD(t),PRES(t),WDIR_ERA5(t),WSPD_ERA5(t),PRES_ERA5(t)
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-12-31 03:00:00,4.0,-0.9,0.6,-8.966873,-0.243606,0.546872,14.0,0.6,0.3,0.785140,...,0.7,4.558071,0.252866,0.613498,-116.0,3.7,0.3,-2.838479,0.622488,0.403992
2022-12-31 04:00:00,14.0,0.6,0.3,0.785140,0.530232,0.788342,-34.0,-3.3,0.7,4.558071,...,0.3,-2.838479,0.622488,0.403992,2.0,2.5,0.4,-7.927598,0.037617,0.105523
2022-12-31 05:00:00,-34.0,-3.3,0.7,4.558071,0.252866,0.613498,-116.0,3.7,0.3,-2.838479,...,0.4,-7.927598,0.037617,0.105523,17.0,-0.9,-0.3,-7.884971,0.040660,-0.078950
2022-12-31 06:00:00,-116.0,3.7,0.3,-2.838479,0.622488,0.403992,2.0,2.5,0.4,-7.927598,...,-0.3,-7.884971,0.040660,-0.078950,5.0,-0.4,-0.5,2.731583,-0.076142,-0.526845
2022-12-31 07:00:00,2.0,2.5,0.4,-7.927598,0.037617,0.105523,17.0,-0.9,-0.3,-7.884971,...,-0.5,2.731583,-0.076142,-0.526845,20.0,0.6,-0.3,5.624592,0.021607,-0.201803


## 2. Transform to supervised problem

In [12]:
#Is already supervised!

## train test split

In [13]:
train_X, train_y, _, _ = DP.train_test_split(train_stationary, -len(train_stationary))
_, _, test_X, test_y = DP.train_test_split(test_stationary, len(test_stationary))
print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

Shapes:  (832867, 3, 6) (832867, 6) (21, 3, 6) (21, 6)


## Normalize the data

In [14]:
# Scale the data
#NORMALIZED = True   #set flag for report

# train_X_scaled, train_y_scaled, test_X_scaled, test_y_scaled, SCALER = DP.scale_data(train_X,
#                                                                                      train_y,
#                                                                                      test_X,
#                                                                                      test_y)

train_X_scaled, train_y_scaled, test_X_scaled, test_y_scaled = train_X, train_y, test_X, test_y #just to exclude scaling for now!

---
# Select Model

In [15]:
#model = models.lstm_0(train_X_scaled,train_y_scaled)
model = Models.get_model(MODEL_NAME, train_X, train_y, ALPHA)
model.summary()

Epoch 1/100


2023-06-27 12:42:43.965438: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


11713/11713 [==============================] - 138s 12ms/step - loss: 550.0145 - val_loss: 547.2686
Epoch 2/100
11713/11713 [==============================] - 95s 8ms/step - loss: 537.8040 - val_loss: 540.3533
Epoch 3/100
11713/11713 [==============================] - 98s 8ms/step - loss: 533.6712 - val_loss: 540.9402
Epoch 4/100
11713/11713 [==============================] - 85s 7ms/step - loss: 529.7054 - val_loss: 535.4293
Epoch 5/100
11713/11713 [==============================] - 83s 7ms/step - loss: 526.3425 - val_loss: 534.5594
Epoch 6/100
11713/11713 [==============================] - 81s 7ms/step - loss: 524.6960 - val_loss: 533.2465
Epoch 7/100
11713/11713 [==============================] - 84s 7ms/step - loss: 524.3285 - val_loss: 533.3201
Epoch 8/100
11713/11713 [==============================] - 82s 7ms/step - loss: 522.8863 - val_loss: 532.0146
Epoch 9/100
11713/11713 [==============================] - 89s 8ms/step - loss: 522.1768 - val_loss: 528.5342
Epoch 10/100
11713/1

KeyboardInterrupt: 

----
# One-Shot Forecasting

In [12]:
_ = model.predict(train_X_scaled, batch_size=1)

108601/108601 [==============================] - 51s 464us/step


In [13]:
output_cols  = test.loc[:, ~test.columns.str.contains('t-')].columns
#output_cols

In [14]:
yhat = model.predict(test_X_scaled)
#yhat

1/1 [==============================] - 0s 439ms/step


In [15]:
#yhat_unscaled = DP.invert_scaling(yhat, SCALER)
yhat_unscaled = yhat # since test_X_scaled is actually not scaled - lol :D
#yhat_unscaled

In [16]:
yhat_unscaled_df = pd.DataFrame(yhat_unscaled)
yhat_unscaled_df.columns = [name + "_pred" for name in output_cols]
yhat_unscaled_df.set_index(test.tail(len(yhat)).index, inplace=True)
#yhat_unscaled_df

In [17]:
evaluation_1 = test.loc[:, ~test.columns.str.contains('t-')]    #ground truth: just y, without X
#evaluation_1

In [18]:
for col in evaluation_1.columns:
    shifted = evaluation_1[col].shift(STATIONARY_SHIFT)
    evaluation_1[f"{col}_pred"]= shifted + yhat_unscaled_df[f"{col}_pred"]

/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_92160/795191291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_1[f"{col}_pred"]= shifted + yhat_unscaled_df[f"{col}_pred"]
/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_92160/795191291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_1[f"{col}_pred"]= shifted + yhat_unscaled_df[f"{col}_pred"]
/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_92160/795191291.py:3: SettingWithCopyWarning: 
A value is

In [19]:
evaluation_1 = evaluation_1.iloc[STATIONARY_SHIFT:]  # remove first n entries since there is no delta value for them
evaluation_1

,WDIR(t),WSPD(t),PRES(t),ATMP(t),WTMP(t),DEWP(t),WDIR_ERA5(t),WSPD_ERA5(t),DEWP_ERA5(t),ATMP_ERA5(t),...,PRES(t)_pred,ATMP(t)_pred,WTMP(t)_pred,DEWP(t)_pred,WDIR_ERA5(t)_pred,WSPD_ERA5(t)_pred,DEWP_ERA5(t)_pred,ATMP_ERA5(t)_pred,PRES_ERA5(t)_pred,WTMP_ERA5(t)_pred
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-12-31 01:00:00,203.0,5.6,1013.8,24.7,24.3,23.5,67.669663,5.587926,21.358772,22.743005,...,1013.692158,24.176418,24.962197,23.440859,-131.275252,4.685999,22.160189,21.726512,1015.335860,24.641998
2022-12-31 02:00:00,169.0,2.3,1014.5,24.8,24.3,24.4,72.227735,5.840792,20.988384,22.496737,...,1014.142189,23.673867,24.960810,22.422370,-130.689406,5.220922,22.038355,21.245548,1016.123503,24.631716
2022-12-31 03:00:00,53.0,6.0,1014.8,22.7,24.4,22.1,69.389256,6.463279,20.995135,23.053178,...,1014.498033,23.780199,24.964378,23.365181,-123.335345,5.464261,21.329049,21.011470,1016.738839,24.654735
2022-12-31 04:00:00,55.0,8.5,1015.2,22.3,24.4,21.9,61.461657,6.500897,21.256585,23.557404,...,1015.392646,21.664081,25.060210,21.009281,-81.343136,6.098771,21.940445,21.557337,1017.148324,24.585931
2022-12-31 05:00:00,72.0,7.6,1014.9,22.6,24.3,22.1,53.576686,6.541557,21.563145,23.751457,...,1015.492778,21.274623,25.060372,20.833599,-142.304334,6.129317,21.884582,22.062655,1017.246734,24.638506
2022-12-31 06:00:00,77.0,7.2,1014.4,23.0,24.4,23.0,56.308269,6.465415,21.715043,23.788865,...,1014.828923,21.580638,24.962938,21.067524,-159.665745,6.159583,21.827037,22.263305,1017.167059,24.664467
2022-12-31 07:00:00,97.0,7.8,1014.1,23.1,24.4,23.0,61.932861,6.487022,21.773962,23.890437,...,1014.254113,21.982142,25.063755,21.980641,-156.083409,6.079706,21.905212,22.304708,1016.641044,24.670462
2022-12-31 08:00:00,148.0,3.3,1013.9,24.4,24.4,24.3,65.444207,6.690521,21.699700,24.076697,...,1014.122998,22.078873,25.062127,21.957771,-149.895539,6.107189,22.131332,22.399892,1016.438410,24.657385
2022-12-31 09:00:00,195.0,5.4,1014.0,25.0,24.4,24.6,60.655439,6.661433,21.758925,24.177490,...,1013.258805,23.388308,25.066284,23.318543,-160.146492,6.289931,21.391914,22.597078,1016.135202,24.697830


# Correct wind direction (modulo 360)

In [ ]:
# Get the list of columns starting with "WDIR"
wdir_columns = [col for col in evaluation_1.columns if col.startswith("WDIR")]

# Modify the values in the selected columns
evaluation_1[wdir_columns] = evaluation_1[wdir_columns] % 360

evaluation_1

In [20]:
# wtmp_true = [col for col in evaluation_1.columns if col.startswith("WTMP")][0]
#
# mae = mean_absolute_error(evaluation_1[wtmp_true], evaluation_1[f"{wtmp_true}_pred"])
# mse = mean_squared_error(evaluation_1[wtmp_true], evaluation_1[f"{wtmp_true}_pred"])
# print('MAE: ', mae)
# print('MSE: ', mse)

MAE:  0.653938645383586
MSE:  0.4379260004114098


In [14]:
#evaluation_1.plot(kind='line')

---
# SAVE

In [22]:
# create a text input widget for username
filename_widget = widgets.Text(
    value='',
    placeholder='Enter filename',
    description='Filename:',
    disabled=False
)
# add '.csv' to the description
extension_label = widgets.Label('.pickle')

# display the widget
display(widgets.HBox([filename_widget, extension_label]))

print("Please also check if the reports description needs to be changed!")

Please also check if the reports description needs to be changed!


In [23]:
report_description="GOM dataset, tested with PINN. Alpha = 0.8"

In [24]:
filename = filename_widget.value

#filename = "report_B_0"


if filename == "":
    print("Enter a valid filename!")

else:
    #Save Data About executed Test:

    #Convert model summary to string
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)

    report = Experiment(
        name=filename,
        description=report_description,

        stations = dataset["files"],
        years = ["not available"],
        nan_threshold=dataset["nan_threshold"],
        features=dataset["features"],
        era5=dataset["add_era5"],

        stationary_shift=STATIONARY_SHIFT,
        # lag=1,
        n_test_hours=dataset["num_test_hours"],

        #stationary=STATIONARY,
        scaler=SCALER,

        model_name = MODEL_NAME,
        model_summary=model_summary,

        one_shot_forecast = evaluation_1,
        recursive_forecast = None
    )


    # open a file for writing in binary mode
    filepath = f'data/reports/{filename}.pickle'
    with open(filepath, 'wb') as f:
        # write the object to the file using pickle.dump()
        pickle.dump(report, f)
        print("File successfully saved:")
        print(filepath)

File successfully saved:
data/reports/GOM_1_B_PINN_08.pickle
